In [2]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Azure AI Search tool from the Azure Agents service using a synchronous client.
    To learn how to set up an Azure AI Search resource,
    visit https://learn.microsoft.com/azure/search/search-get-started-portal

USAGE:
    Before running the sample:
    Set these environment variables with your own values:
    1) AZURE_AISTUDIO_PROJECT_CONN_STRING - The project connection string, as found in the overview page of your
       Azure AI Foundry project.
    2) AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in 
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_AI_SEARCH_CONNECTION_ID - The connection ID of the Azure AI Search resource, as found in the "Connections" tab
"""
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential
from typing import Any, Callable, Set, Dict, List, Optional
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
from azure.identity import DefaultAzureCredential

load_dotenv()
AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = "gpt-4o"
AZURE_AISTUDIO_PROJECT_CONN_STRING = os.getenv("NL_TO_KQL_AZURE_AISTUDIO_PROJECT_CONN_STRING")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=AZURE_AISTUDIO_PROJECT_CONN_STRING,
)


try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    print(ex)

ImportError: cannot import name 'CodeInterpreterTool' from 'azure.ai.projects.models' (c:\Users\dschlesinger\code\ongoing\genai-agents\.venv\lib\site-packages\azure\ai\projects\models\__init__.py)

In [ ]:
#Define a function to be called by the agent as a custom tool
import json

def recent_snowfall(location: str) -> str:
    """
    Fetches recent snowfall totals for a given location.
    :param location: The city name.
    :return: Snowfall details as a JSON string.
    """
    mock_snow_data = {"Seattle": "0 inches", "Denver": "2 inches"}
    snow = mock_snow_data.get(location, "Data not available.")
    return json.dumps({"location": location, "snowfall": snow})

user_functions: Set[Callable[..., Any]] = {
    recent_snowfall,
}

In [ ]:
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# Create your agent with the toolset
agent = project_client.agents.create_agent(
    model=AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME,
    name="snowfall-agent",
    instructions="You are a weather assistant tracking snowfall. Use the provided functions to answer questions.",
    toolset=toolset
)

In [ ]:
# Create thread for communication
def call_agent(question):
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
        # Fetch and log all messages
        
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # print(messages["data"][0]["content"][0]["text"]["value"])
    # Fetch and log all messages in chronological order
    messages_data = messages["data"]

    # Sort messages by creation time (ascending)
    sorted_messages = sorted(messages_data, key=lambda x: x["created_at"])

    print("\n--- Thread Messages (sorted) ---")
    for msg in sorted_messages:
        role = msg["role"].upper()
        # Each 'content' is a list; get the first text block if present
        content_blocks = msg.get("content", [])
        text_value = ""
        if content_blocks and content_blocks[0]["type"] == "text":
            text_value = content_blocks[0]["text"]["value"]
        print(f"{role}: {text_value}")

In [ ]:
# Send a prompt to the agent
user_prompt = "What is the recent snowfall in Seattle?"

call_agent(user_prompt)

Created thread, ID: thread_mG5qqMPufjWVNvSWmhn7ZOIy
Created message, ID: msg_SmCyQ37LiZnBKuMFeciaKt91
Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_hg4vtpNg5oKXT8LAGgBzGu2d', 'object': 'thread.message', 'created_at': 1743241266, 'assistant_id': 'asst_LMKRzIYi3ICl96x38StdlAZE', 'thread_id': 'thread_mG5qqMPufjWVNvSWmhn7ZOIy', 'run_id': 'run_nqlSkxThpePVOUgvYXzqQjpW', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The recent snowfall in Seattle is 0 inches.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_SmCyQ37LiZnBKuMFeciaKt91', 'object': 'thread.message', 'created_at': 1743241261, 'assistant_id': None, 'thread_id': 'thread_mG5qqMPufjWVNvSWmhn7ZOIy', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is the recent snowfall in Seattle?', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_hg4vtpNg5oKXT8LAGgBzGu2d', 'last_id': 'msg_SmCyQ37LiZnBKuM